In [ ]:
# Instalação (caso necessário) — o Google Colab já traz a maioria
!pip -q install networkx scikit-learn pandas matplotlib

## Configurações do Cenário
Ajuste estes parâmetros para explorar diferentes condições (por exemplo, hora do almoço com mais tráfego).

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from math import sqrt
import random

np.random.seed(42)
random.seed(42)

PARAMS = {
    'grid_rows': 10,          # Tamanho da grade (linhas)
    'grid_cols': 10,          # Tamanho da grade (colunas)
    'edge_base_time': 1.0,    # tempo base por aresta (minutos) antes do tráfego
    'traffic_low': 0.8,       # fator mínimo de tráfego
    'traffic_high': 1.6,      # fator máximo de tráfego
    'n_deliveries': 25,       # quantidade de pedidos (pontos de entrega)
    'n_clusters': 3,          # número de zonas/entregadores (K-Means)
    'km_per_grid': 0.20,      # cada passo na grade ≈ 0.20 km
    'avg_speed_kmh': 18,      # velocidade média urbana (km/h)
    'fuel_l_per_km': 0.07,    # consumo médio (litros por km)
    'fuel_price': 6.50,       # preço estimado por litro (R$)
}

PARAMS

## 1) Cidade como Grafo
Vamos criar uma cidade em **grade** (10x10). Cada **nó** é uma intersecção `(linha, coluna)` e cada **aresta** conecta vizinhos (cima/baixo/esquerda/direita). O **peso** da aresta representa o tempo (em minutos), incluindo um fator de tráfego aleatório para simular horários de pico.

In [ ]:
def build_city_graph(rows, cols, base_time, traffic_low, traffic_high):
    G = nx.Graph()
    # adiciona nós com coordenadas (para visualização e heurística)
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c), pos=(c, -r))  # y negativo só para desenhar “de cima para baixo”
    
    # adiciona arestas com peso = tempo (minutos) ajustado por tráfego
    for r in range(rows):
        for c in range(cols):
            if r + 1 < rows:
                traffic = np.random.uniform(traffic_low, traffic_high)
                G.add_edge((r, c), (r+1, c), weight=base_time * traffic)
            if c + 1 < cols:
                traffic = np.random.uniform(traffic_low, traffic_high)
                G.add_edge((r, c), (r, c+1), weight=base_time * traffic)
    return G

G = build_city_graph(
    PARAMS['grid_rows'], PARAMS['grid_cols'],
    PARAMS['edge_base_time'], PARAMS['traffic_low'], PARAMS['traffic_high']
)
len(G.nodes()), len(G.edges())

### Visualização do Grafo da Cidade
Os nós representam intersecções. *Não definimos cores específicas*, apenas a estrutura e os marcadores padrão.

In [ ]:
plt.figure(figsize=(6, 6))
pos = nx.get_node_attributes(G, 'pos')
nx.draw(G, pos, node_size=30, with_labels=False)
plt.title('Grafo da Cidade (Grade)')
plt.show()

## 2) Pedidos e Restaurante (Origem)
- Restaurante no **centro** da grade.
- Pedidos são pontos aleatórios (nós da grade).

In [ ]:
rows, cols = PARAMS['grid_rows'], PARAMS['grid_cols']
origin = (rows // 2, cols // 2)

all_nodes = list(G.nodes())
deliveries = random.sample([n for n in all_nodes if n != origin], PARAMS['n_deliveries'])

df_del = pd.DataFrame(deliveries, columns=['r', 'c'])
df_del.head()

### Visualização: Origem e Pedidos

In [ ]:
plt.figure(figsize=(6, 6))
nx.draw(G, pos, node_size=10, with_labels=False)
x_origin, y_origin = pos[origin]
plt.scatter([x_origin], [y_origin], s=120, marker='*')
xs = [pos[(r,c)][0] for r,c in deliveries]
ys = [pos[(r,c)][1] for r,c in deliveries]
plt.scatter(xs, ys, s=40, marker='o')
plt.title('Restaurante (★) e Pedidos (o)')
plt.show()

## 3) Algoritmo A* (A-estrela)
Usaremos A* com **heurística Manhattan** (distância em grade) para encontrar caminhos mais rápidos entre dois nós. O peso da aresta já considera o tráfego simulado.

In [ ]:
def manhattan(u, v):
    (r1, c1), (r2, c2) = u, v
    return abs(r1 - r2) + abs(c1 - c2)

def astar_path(G, source, target):
    return nx.astar_path(G, source, target, heuristic=manhattan, weight='weight')

def path_time(G, path):
    return sum(G[u][v]['weight'] for u, v in zip(path[:-1], path[1:]))

# Teste rápido com o primeiro pedido
sample_target = deliveries[0]
p = astar_path(G, origin, sample_target)
path_time(G, p)

## 4) Rota Multi-Paradas (Heurística Vizinho Mais Próximo)
Problema real é visitar **vários** destinos. Usamos uma heurística simples: sempre ir ao próximo destino com **menor tempo A*** a partir do ponto atual.

> *Observação:* Resolver exatamente é um problema complexo (TSP). A heurística dá uma **boa aproximação** com custo computacional baixo.

In [ ]:
def nearest_neighbor_route(G, origin, targets):
    remaining = set(targets)
    current = origin
    full_path = []
    total_time = 0.0
    visit_order = []
    
    while remaining:
        # escolhe o próximo destino com menor tempo A*
        best = None
        best_time = float('inf')
        best_path = None
        for t in remaining:
            p = astar_path(G, current, t)
            ttime = path_time(G, p)
            if ttime < best_time:
                best = t
                best_time = ttime
                best_path = p
        # atualiza
        if full_path:
            full_path.extend(best_path[1:])
        else:
            full_path.extend(best_path)
        total_time += best_time
        visit_order.append(best)
        current = best
        remaining.remove(best)
    return full_path, total_time, visit_order

route_nn_path, route_nn_time, route_nn_order = nearest_neighbor_route(G, origin, deliveries)
route_nn_time

### Visualização: Rota com Heurística do Vizinho Mais Próximo

In [ ]:
plt.figure(figsize=(6, 6))
nx.draw(G, pos, node_size=10, with_labels=False)
# desenha a rota destacando as arestas do caminho encontrado
path_edges = list(zip(route_nn_path[:-1], route_nn_path[1:]))
nx.draw_networkx_nodes(G, pos, nodelist=[origin], node_size=120, node_shape='*')
nx.draw_networkx_nodes(G, pos, nodelist=deliveries, node_size=40, node_shape='o')
nx.draw_networkx_edges(G, pos, edgelist=path_edges, width=2.5)
plt.title('Rota — Vizinho Mais Próximo (A*)')
plt.show()

## 5) Agrupamento de Entregas (K-Means)
Agrupamos os pedidos em **K** zonas. A ideia é atribuir cada zona para um **entregador** diferente, reduzindo deslocamentos totais.

In [ ]:
coords = df_del[['r','c']].values
k = PARAMS['n_clusters']
kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
labels = kmeans.fit_predict(coords)
df_del['cluster'] = labels
df_del.head()

### Visualização: Clusters (marcadores diferentes por zona)
Para seguir a regra de **não definir cores específicas**, usamos **marcadores diferentes** para cada cluster.

In [ ]:
markers = ['o','s','^','D','P','X','v','<','>','h']
plt.figure(figsize=(6, 6))
nx.draw(G, pos, node_size=10, with_labels=False)
for cl in sorted(df_del['cluster'].unique()):
    sub = df_del[df_del['cluster']==cl]
    xs = [pos[(r,c)][0] for r,c in sub[['r','c']].itertuples(index=False, name=None)]
    ys = [pos[(r,c)][1] for r,c in sub[['r','c']].itertuples(index=False, name=None)]
    plt.scatter(xs, ys, s=60, marker=markers[cl % len(markers)], label=f'Zona {cl}')
x0,y0 = pos[origin]
plt.scatter([x0],[y0], s=120, marker='*', label='Restaurante')
plt.legend()
plt.title('Pedidos Agrupados em Zonas (K-Means)')
plt.show()

## 6) Rotas por Zona (um entregador por cluster)
Para cada cluster, calculamos a rota por **Vizinho Mais Próximo + A***.
Em seguida, somamos os tempos para comparar com uma rota única.

In [ ]:
cluster_routes = {}
total_time_clusters = 0.0
for cl in sorted(df_del['cluster'].unique()):
    points = [tuple(x) for x in df_del[df_del['cluster']==cl][['r','c']].values]
    rpath, rtime, order = nearest_neighbor_route(G, origin, points)
    cluster_routes[cl] = {'path': rpath, 'time': rtime, 'order': order}
    total_time_clusters += rtime

total_time_clusters

### Visualização: Rotas por Zona
Desenhamos cada rota de cluster com **estilos de linha diferentes** para distinção (sem definir cores manualmente).

In [ ]:
linestyles = ['-', '--', ':', '-.']
plt.figure(figsize=(6, 6))
nx.draw(G, pos, node_size=10, with_labels=False)
nx.draw_networkx_nodes(G, pos, nodelist=[origin], node_size=120, node_shape='*')
for cl, data in cluster_routes.items():
    pe = list(zip(data['path'][:-1], data['path'][1:]))
    nx.draw_networkx_edges(G, pos, edgelist=pe, width=2.2, style=linestyles[cl % len(linestyles)])
plt.title('Rotas por Zona (um entregador por cluster)')
plt.show()

## 7) Baseline: Ordem Aleatória
Para ter uma referência simples (*baseline*), calculamos o tempo total seguindo uma **ordem aleatória** dos pedidos (ainda usando A* entre cada par).

In [ ]:
def random_route_time(G, origin, targets):
    order = list(targets)
    random.shuffle(order)
    current = origin
    ttotal = 0.0
    for t in order:
        p = astar_path(G, current, t)
        ttotal += path_time(G, p)
        current = t
    return ttotal, order

baseline_time, baseline_order = random_route_time(G, origin, deliveries)
baseline_time

## 8) Métricas e Comparações
Transformamos o tempo em **distância aproximada** e **custo de combustível**.

- Cada passo na grade ≈ `km_per_grid` km.
- Estimamos a distância a partir do tempo e da velocidade média.
- Custo = `distância * fuel_l_per_km * fuel_price`.

In [ ]:
def time_to_distance_km(minutes, avg_speed_kmh):
    hours = minutes / 60.0
    return hours * avg_speed_kmh

def fuel_cost(dist_km, l_per_km, price):
    return dist_km * l_per_km * price

t_nn = route_nn_time
t_clusters = total_time_clusters
t_base = baseline_time

d_nn = time_to_distance_km(t_nn, PARAMS['avg_speed_kmh'])
d_clusters = time_to_distance_km(t_clusters, PARAMS['avg_speed_kmh'])
d_base = time_to_distance_km(t_base, PARAMS['avg_speed_kmh'])

c_nn = fuel_cost(d_nn, PARAMS['fuel_l_per_km'], PARAMS['fuel_price'])
c_clusters = fuel_cost(d_clusters, PARAMS['fuel_l_per_km'], PARAMS['fuel_price'])
c_base = fuel_cost(d_base, PARAMS['fuel_l_per_km'], PARAMS['fuel_price'])

compare = pd.DataFrame({
    'Cenário': ['Baseline (ordem aleatória)', 'Uma rota (Vizinho+ A*)', 'Zonas (K-Means + rotas)'],
    'Tempo Total (min)': [t_base, t_nn, t_clusters],
    'Distância Est. (km)': [d_base, d_nn, d_clusters],
    'Custo Combustível (R$)': [c_base, c_nn, c_clusters]
})
compare

In [ ]:
import matplotlib.ticker as mtick

plt.figure(figsize=(7,4))
plt.bar(compare['Cenário'], compare['Tempo Total (min)'])
plt.title('Tempo Total por Cenário (min)')
plt.xticks(rotation=20)
plt.ylabel('Minutos')
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,4))
plt.bar(compare['Cenário'], compare['Custo Combustível (R$)'])
plt.title('Custo Estimado de Combustível por Cenário (R$)')
plt.xticks(rotation=20)
plt.ylabel('R$')
plt.tight_layout()
plt.show()

## 9) Tabela: Ordem de Visita
Mostramos a sequência de entregas para os dois métodos principais.

In [ ]:
order_nn_df = pd.DataFrame(route_nn_order, columns=['r','c'])
cluster_order_rows = []
for cl, data in cluster_routes.items():
    for (r,c) in data['order']:
        cluster_order_rows.append({'cluster': cl, 'r': r, 'c': c})
order_clusters_df = pd.DataFrame(cluster_order_rows)
order_nn_df.head(), order_clusters_df.head()

## 10) Conclusões e Próximos Passos
**Resultados desta simulação**
- A rota com **Vizinho Mais Próximo + A*** já reduz tempo em relação ao baseline (ordem aleatória).
- Ao **dividir em zonas (K-Means)** e otimizar cada rota, o **tempo total** tende a cair ainda mais quando vários entregadores atuam simultaneamente.

**Próximos passos práticos**
1. Incorporar **janelas de tempo** (clientes com horários específicos).
2. Considerar **capacidade** do entregador (peso/volume) e pausas.
3. Integrar **tráfego em tempo real** (dados reais da cidade).
4. Trocar a heurística por **metaheurísticas** (ex.: Simulated Annealing, VNS) para rotas ainda melhores.
5. Integrar com **app mobile** para navegação e check-in de entregas.

_Dica para apresentação_: Mostre os gráficos, compare as barras de tempo/custo e explique a intuição: “dividir para conquistar” + “sempre pegar o próximo mais rápido”.

---
### (Opcional) Reexecutar com Novas Sementes
Para mudar os resultados, altere a semente aleatória e rode as células de novo.

In [ ]:
def reseed(seed=123):
    np.random.seed(seed)
    random.seed(seed)
    print('Sementes redefinidas:', seed)

reseed(123)